In [1]:
%cd "fewshot-face-translation-GAN"

/home/alee/Documents/MuhammadAlee/FaceSwap/fewshot-face-translation-GAN


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from models import FaceTranslationGANInferenceModel

Using TensorFlow backend.


In [4]:
model = FaceTranslationGANInferenceModel()













Found checkpoints in weights folder. Built model with pre-trained weights.


In [5]:
from face_toolbox_keras.models.verifier.face_verifier import FaceVerifier
fv = FaceVerifier(classes=512)

from face_toolbox_keras.models.parser import face_parser
fp = face_parser.FaceParser()

from face_toolbox_keras.models.detector import face_detector
fd = face_detector.FaceAlignmentDetector()

from face_toolbox_keras.models.detector.iris_detector import IrisDetector
idet = IrisDetector()
#idet.set_detector(fd)
from utils.faceswap import FaceSwap
fs=FaceSwap()




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [6]:
import numpy as np
from utils import utils
from matplotlib import pyplot as plt

In [7]:
!ffmpeg -i Video/src/cut.mp4 Video/raw_images/$filename%03d.png

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [8]:
def correct_colours(im1, im2):
    COLOUR_CORRECT_BLUR_FRAC=0.6
    blur_amount = COLOUR_CORRECT_BLUR_FRAC * np.linalg.norm(
                              np.mean(im1, axis=0) -
                              np.mean(im2, axis=0))
    blur_amount = int(blur_amount)
    if blur_amount % 2 == 0:
        blur_amount += 1
    im1_blur = cv2.GaussianBlur(im1, (blur_amount, blur_amount), 0)
    im2_blur = cv2.GaussianBlur(im2, (blur_amount, blur_amount), 0)

    # Avoid divide-by-zero errors.
    im2_blur += (128 * (im2_blur <= 1.0)).astype(im2_blur.dtype)

    result= (im2.astype(np.float64) * im1_blur.astype(np.float64) /
                                                im2_blur.astype(np.float64))
    result = np.clip(result, 0, 255).astype(np.uint8)
    return result

In [12]:
import os
import os.path
import shutil
from PIL import Image
import cv2


target_path = 'Video/src/ik.jpg'
tar, emb_tar = utils.get_tar_inputs([target_path], fd, fv)
src_dir = "Video/raw_images/"
result_dir = "Video/results/"
os.makedirs(result_dir, exist_ok=True)

for target_name in os.listdir(src_dir):
  src_path = os.path.join(src_dir, target_name)
  result_path = os.path.join(result_dir, target_name)
  print(src_path)
  try:
    src, mask, aligned_im, (x0, y0, x1, y1), landmarks,M = utils.get_src_inputs(src_path, fd, fp, idet)
  except IndexError as e:
    print("get_src_inputs IndexError:", e)
    shutil.copyfile(src_path, result_path)
    continue
  except AssertionError as e:
    print("get_src_inputs AssertionError:", e)
    shutil.copyfile(src_path, result_path)
    continue

  tar=cv2.resize(tar, (src.shape[1], src.shape[0]))
  tar=correct_colours(src,tar)
  out = model.inference(src, mask, tar, emb_tar)
  result_face = np.squeeze(((out[0] + 1) * 255 / 2).astype(np.uint8))
  result_face=cv2.resize(result_face, (src.shape[1], src.shape[0]))
  rst=result_face
  trs=fs.transform(src,result_face)

  if type(trs)!=int:    
       result_face=trs
       mouth=cv2.imread("mouth.png")
       a,b = np.where(np.all(mouth!=[0,0,0],axis=2))
       src=correct_colours(trs,src)
       result_face[a,b]=src[a,b]
       a,b = np.where(np.all(mask==[255, 75, 125],axis=2))
       result_face[a,b]=rst[a,b]
       blur = cv2.GaussianBlur(result_face,(5,5),0)
       a,b = np.where(np.all(mask==[0,0,255],axis=2))
       result_face[a,b]=blur[a,b]
    
    
  result_img = utils.post_process_result(src_path, fd, result_face, aligned_im, src, x0, y0, x1, y1, landmarks,M)
  Image.fromarray(result_img).save(result_path)

Video/raw_images/280.png
Video/raw_images/040.png
Multiple faces detected. Only the most confident face will be processed.
Video/raw_images/106.png
Multiple faces detected. Only the most confident face will be processed.
Multiple faces detected. Only the most confident face will be processed.
Video/raw_images/046.png
Multiple faces detected. Only the most confident face will be processed.
Video/raw_images/366.png
Video/raw_images/257.png
Video/raw_images/412.png
Video/raw_images/264.png
Video/raw_images/402.png
Video/raw_images/168.png
Video/raw_images/252.png
Video/raw_images/345.png
Video/raw_images/333.png
Video/raw_images/312.png
Video/raw_images/409.png
Video/raw_images/042.png
Multiple faces detected. Only the most confident face will be processed.
Multiple faces detected. Only the most confident face will be processed.
Video/raw_images/359.png
Video/raw_images/173.png
Video/raw_images/117.png
Multiple faces detected. Only the most confident face will be processed.
Video/raw_imag

Video/raw_images/065.png
Multiple faces detected. Only the most confident face will be processed.
Multiple faces detected. Only the most confident face will be processed.
Video/raw_images/298.png
Video/raw_images/198.png
Video/raw_images/092.png
Multiple faces detected. Only the most confident face will be processed.
Video/raw_images/186.png
Video/raw_images/297.png
Video/raw_images/165.png
Video/raw_images/394.png
Video/raw_images/158.png
Multiple faces detected. Only the most confident face will be processed.
Video/raw_images/007.png
Multiple faces detected. Only the most confident face will be processed.
Multiple faces detected. Only the most confident face will be processed.
Video/raw_images/062.png
Multiple faces detected. Only the most confident face will be processed.
Multiple faces detected. Only the most confident face will be processed.
Video/raw_images/396.png
Video/raw_images/334.png
Video/raw_images/011.png
Multiple faces detected. Only the most confident face will be proc

Multiple faces detected. Only the most confident face will be processed.
Multiple faces detected. Only the most confident face will be processed.
Video/raw_images/434.png
Video/raw_images/292.png
Video/raw_images/056.png
Multiple faces detected. Only the most confident face will be processed.
Multiple faces detected. Only the most confident face will be processed.
Video/raw_images/322.png
Video/raw_images/362.png
Video/raw_images/074.png
Multiple faces detected. Only the most confident face will be processed.
Multiple faces detected. Only the most confident face will be processed.
Video/raw_images/318.png
Video/raw_images/290.png
Video/raw_images/116.png
Multiple faces detected. Only the most confident face will be processed.
Multiple faces detected. Only the most confident face will be processed.
Video/raw_images/291.png
Video/raw_images/093.png
Multiple faces detected. Only the most confident face will be processed.
Multiple faces detected. Only the most confident face will be proces

Multiple faces detected. Only the most confident face will be processed.
Video/raw_images/447.png
Video/raw_images/169.png
Video/raw_images/356.png
Video/raw_images/313.png
Video/raw_images/044.png
Multiple faces detected. Only the most confident face will be processed.
Video/raw_images/089.png
Multiple faces detected. Only the most confident face will be processed.
Multiple faces detected. Only the most confident face will be processed.
Video/raw_images/090.png
Multiple faces detected. Only the most confident face will be processed.
Multiple faces detected. Only the most confident face will be processed.
Video/raw_images/240.png
Video/raw_images/094.png
Multiple faces detected. Only the most confident face will be processed.
Multiple faces detected. Only the most confident face will be processed.
Video/raw_images/197.png
Video/raw_images/105.png
Multiple faces detected. Only the most confident face will be processed.
Multiple faces detected. Only the most confident face will be proces

In [14]:
!ffmpeg -f image2 -i Video/results/$filename%03d.png  -vcodec libx264 -pix_fmt yuv420p -strict -2 -acodec aac video.mp4 -y

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib